<a href="https://colab.research.google.com/github/murilo-henrique060/matematica-computacional/blob/main/Avalia%C3%A7%C3%A3o%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np

# Busca de Raízes com o método da bisseção

Encontrar raízes de funções reais é uma técnica muito utilizada no meio da programação. Essa técnica é separada em 2 partes:
- Isolamento das raízes: Determinar intervalos que contenham uma raiz;
- Refinamento: Utilização de um algoritmo, como a bisseção, para determinar um valor aproximado para a raiz de cada intervalo encontrado.

Vamos utilizar de exemplo a função:
$$
f(x) = x^3 - 9x + 3
$$

## Isolamento de raízes

Para realizar o isolamento de raízes, podemos utilizar o Teorema do Valor Intermediário, que diz que, para uma função contínua $f(x)$ em um intervalo fechado $[a, b]$, se temos um valor $k$ entre $f(a)$ e $f(b)$, então temos, pelo menos, um ponto $c$ tal que $f(c)$ = k.

Baseado no teorema, podemos buscar intervalos $[a, b]$, tal que $f(a) > 0$ e $f(b) < 0$, ou que $f(a) < 0$ e $f(b) > 0$, ou seja, $f(a)$ e $f(b)$ devem ter sinais opostos.

Para simplificar a definição, podemos buscar pontos $a$ e $b$ tal que $f(a) \cdot f(b) < 0$.

### Método Iterativo

Para isolar as raízes, podemos definir um intervalo de busca e um passo, assim podemos separar o intervalo em seções para realizar a busca dos intervalos

In [12]:
def root_isolation_basic(f, start=-10, end=10, sections=4, verbose=False):
  """
  Search for intervals that have at least a root in it
  """
  intervals = []

  # Defining the points inside the interval
  points = np.linspace(start, end, sections)

  # Iterating each interval (each 2 adjacent points define an interval)
  for a, b in zip(points, points[1:]):
    if verbose:
      print(f"x: {a:>5.1f}  f(x): {'-' if f(a) < 0 else '+'}")

    # If the signal of f(a) and f(b) are different there is, at least, one root inside the interval
    if f(a) * f(b) <= 0:
      intervals.append((float(a), float(b)))

  if verbose:
    print(f"x: {b:>5.1f}  f(x): {'-' if f(b) < 0 else '+'}")

  return intervals

f = lambda x: x**3 - 9 * x + 3 # x³ - 9x + 3
intervals = list(root_isolation_basic(f, verbose=True))

print()

print("Intervalos: " + " ".join(f"[{x}, {x1}]" for x, x1 in intervals))

x: -10.0  f(x): -
x:  -3.3  f(x): -
x:   3.3  f(x): +
x:  10.0  f(x): +

Intervalos: [-3.333333333333333, 3.333333333333334]


### Checagem de raízes

Para verificar se o intervalo contém apenas uma única raiz, podemos checar o sinal da derivada ao longo do intervalo, caso seja constante, confirmamos que há apenas uma derivada neste intervalo.

Para o exemplo apresentado teremos a derivada como:
$$
f'(x) = 3x^2 - 9
$$


In [38]:
def check_unique_root(d, a, b, samples=10):
  """
  Function that checks if there is only a single root inside the interval
  """

  # Defining the points interval
  points = np.linspace(a, b, samples)

  # Calculating the derivative of the first point
  d_a = d(a)

  # Iterating the points
  for x in points:
    # If the signal of the derivative changes in any point, so the interval can have more than 1 root on the interval
    if (d_a * d(x) < 0):
      return False

  return True

d = lambda x: 3 * x**2 - 9 # 3x² - 9
intervals = list(root_isolation_basic(f))
print()

print("Intervalos: " + " ".join(f"[{x}, {x1}]" for x, x1 in intervals))

print()
for a, b in intervals:
  print(f"[{a}, {b}] -> {'Tem raiz única' if check_unique_root(d, a, b) else 'Contém mais de uma raiz'}")


Intervalos: [-3.333333333333333, 3.333333333333334]

[-3.333333333333333, 3.333333333333334] -> Contém mais de uma raiz


Então, no caso do intervalo conter mais de uma raiz, podemos realizar uma busca mais refinada para encontrar raízes únicas.

In [19]:
def root_isolation(f, d, start=-10, end=10, samples=4):
  """
  Return all the single root subintervals inside the interval passed
  """
  intervals = []

  # Get all intervals that have 1 or more roots in it
  possible_intervals = root_isolation_basic(f, start, end, samples)

  # Iterating the intervals
  for a, b in possible_intervals:
    # If the interval has only one root, add it to the result
    if check_unique_root(d, a, b):
      intervals.append((a, b))

    # If not, compute all the single root subintervals inside the interval
    else:
      intervals.extend(root_isolation(f, d, a, b))

  return intervals

intervals = root_isolation(f, d)
print()

print("Intervalos: " + " ".join(f"[{x}, {x1}]" for x, x1 in intervals))


Intervalos: [-3.333333333333333, -2.592592592592592] [-1.1111111111111107, 1.1111111111111116] [2.5925925925925934, 3.333333333333334]


## Refinamento das Raízes

Para refinar as raízes utilizaremos o método da bisseção, no qual iniciamos com o intervalo $[a, b]$, e encontramos o ponto médio $x = \frac{(a + b)}{2}$, e verificamos se $f(x) * f(a) >= 0$, caso seja, $a$ recebe o valor de $x$, caso não, $b$ recebe o valor de $x$. Repetimos esse processo até que $b - a < ϵ$, para um $ϵ$ definido.

In [36]:
def bissect(function, a, b, e=10**-3, verbose=True):
  """
  Returns a aproximation of the root location inside the interval where one side result is greater than 0 and the other one is smaller
  """
  f_a = function(a)

  if verbose:
    print(f"| Iteration |     a     |     b     |   b - a   |     x     |    f(x)   |")

  i = 0
  # The computation stops when the distante of the 2 points is smaller than the e defined
  while b - a > e:
    # Calculating the middle point
    x = (a + b) / 2
    # Computing the f(x) value
    f_x = function(x)

    if verbose:
      print(f"| {i:^9} | {a:>9.5f} | {b:>9.5f} | {b-a:>9.5f} | {x:>9.5f} | {f_x:>9.5f} |")

    # If the signal of f(x) is the same as f(a), so update the value of a to x
    if f_a * f_x >= 0:
      a = x
      f_a = f_x

    # If not, update the value of b to x
    else:
      b = x

    i += 1

  if verbose:
      print(f"| {i:^9} | {a:>9.5f} | {b:>9.5f} | {b-a:>9.5f} | {x:>9.5f} | {f_x:>9.5f} |\n")

  return x

roots = [bissect(f, a, b) for a, b in root_isolation(f, d)]
print("Raízes:", roots)

| Iteration |     a     |     b     |   b - a   |     x     |    f(x)   |
|     0     |  -3.33333 |  -2.59259 |   0.74074 |  -2.96296 |   3.65437 |
|     1     |  -3.33333 |  -2.96296 |   0.37037 |  -3.14815 |   0.13255 |
|     2     |  -3.33333 |  -3.14815 |   0.18519 |  -3.24074 |  -1.86889 |
|     3     |  -3.24074 |  -3.14815 |   0.09259 |  -3.19444 |  -0.84763 |
|     4     |  -3.19444 |  -3.14815 |   0.04630 |  -3.17130 |  -0.35244 |
|     5     |  -3.17130 |  -3.14815 |   0.02315 |  -3.15972 |  -0.10868 |
|     6     |  -3.15972 |  -3.14815 |   0.01157 |  -3.15394 |   0.01225 |
|     7     |  -3.15972 |  -3.15394 |   0.00579 |  -3.15683 |  -0.04813 |
|     8     |  -3.15683 |  -3.15394 |   0.00289 |  -3.15538 |  -0.01792 |
|     9     |  -3.15538 |  -3.15394 |   0.00145 |  -3.15466 |  -0.00283 |
|    10     |  -3.15466 |  -3.15394 |   0.00072 |  -3.15466 |  -0.00283 |

| Iteration |     a     |     b     |   b - a   |     x     |    f(x)   |
|     0     |  -1.11111 |   1.11111 |

In [ ]:
def binary_to_fractions(bits, fractions=0):
  return sum(int(bit) * 2**(power) for power, bit in enumerate(reversed(bits), start=-fractions))

In [ ]:
def integer_to_binary(value, places=8):
  result = ""

  while (value > 0 and len(result) < places):
    value, bit = divmod(value, 2)
    result = str(bit) + result

  return f"{result:>0{places}}"

def fractional_to_binary(value, places=8):
  result = ""

  while (value > 0 and len(result) < places):
    value *= 2
    bit = int(value)
    value -= bit

    result += str(bit)

  return f"{result:<0{places}}"

def to_binary(value, signed=False, integral_places=8, fractional_places=8):
  signal = ""
  if signed:
    signal = "0" if value >= 0 else "1"

  integral, fractional = divmod(abs(value), 1)
  integral = int(integral)

  return signal + integer_to_binary(integral, integral_places) + fractional_to_binary(fractional, fractional_places)

def format_binary(bits, signed=False, integral_places=8, fractional_places=8):
  signal = ""
  if signed:
    signal = "+" if bits[0] == "0" else "-"
    bits = bits[1:]

  integer = bits[:integral_places]
  fractional = bits[-fractional_places:]

  return signal + f"{integer.lstrip('0'):>01}" +  "." + f"{fractional.rstrip('0'):<01}"

format_binary(to_binary(12.3))

In [ ]:
def float32_to_binary(bits):
  bias = 127

  signal = parse_binary_string(bits[0])
  biased_expoent = parse_binary_string(bits[1:9])
  mantissa = parse_binary_string(bits[9:], fractional=23)

  value = (-1)**signal * (1 + mantissa) * 2**(biased_expoent - bias)

  return to_binary(value)

bin_val = float32_to_binary("00111110010100000000000000000000")
print(bin_val)
print(format_binary(bin_val))
dec_val = binary_to_fractions(bin_val, 8)
print(dec_val)